In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import pandas as pd
import numpy as np

import os
TRAIN_ABLE_FALSE=True
if TRAIN_ABLE_FALSE:
    os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import sklearn.metrics as mtr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
import time

from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda,BatchNormalization,LeakyReLU,PReLU,ELU,ThresholdedReLU,Concatenate
from keras.models import Model
import keras.backend as K
from keras.callbacks import Callback
from  keras.callbacks import EarlyStopping,ModelCheckpoint
import datetime
import warnings
warnings.filterwarnings('ignore')

#TRAIN_OFFLINE = False



In [0]:
train_df = pd.read_csv("/content/gdrive/My Drive/nfl_eda_2.csv")

In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23171 entries, 0 to 23170
Data columns (total 34 columns):
Unnamed: 0                  23171 non-null int64
GameId                      23171 non-null int64
PlayId                      23171 non-null int64
back_from_scrimmage         23171 non-null float64
back_oriented_down_field    23171 non-null int64
back_moving_down_field      23171 non-null int64
min_dist                    23171 non-null float64
max_dist                    23171 non-null float64
mean_dist                   23171 non-null float64
std_dist                    23171 non-null float64
def_min_dist                23171 non-null float64
def_max_dist                23171 non-null float64
def_mean_dist               23171 non-null float64
def_std_dist                23171 non-null float64
PlayerHeight_dense          23171 non-null int64
PlayerAge                   23171 non-null float64
WindSpeed_dense             23171 non-null float64
GameWeather_dense           23171 no

In [0]:
train_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [0]:
X = train_df.copy()
yards = X.Yards

y = np.zeros((yards.shape[0], 199))

for idx, target in enumerate(list(yards)):
    y[idx][99 + target] = 1

X.drop(['GameId','PlayId','Yards'], axis=1, inplace=True)

In [0]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [0]:
class CRPSCallback(Callback):
    
    def __init__(self,validation, predict_batch_size=20, include_on_batch=False):
        super(CRPSCallback, self).__init__()
        self.validation = validation
        self.predict_batch_size = predict_batch_size
        self.include_on_batch = include_on_batch
        
        print('validation shape',len(self.validation))

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_train_begin(self, logs={}):
        if not ('CRPS_score_val' in self.params['metrics']):
            self.params['metrics'].append('CRPS_score_val')

    def on_batch_end(self, batch, logs={}):
        if (self.include_on_batch):
            logs['CRPS_score_val'] = float('-inf')

    def on_epoch_end(self, epoch, logs={}):
        logs['CRPS_score_val'] = float('-inf')
            
        if (self.validation):
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_pred = self.model.predict(X_valid)
            y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
            y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
            val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
            val_s = np.round(val_s, 6)
            logs['CRPS_score_val'] = val_s

In [0]:
# Calculate CRPS score
def crps_score(y_prediction, y_valid, shape=X.shape[0]):
    y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_prediction, axis=1), 0, 1)
    val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * shape)
    crps = np.round(val_s, 6)
    
    return crps

In [0]:
from sklearn.ensemble import RandomForestRegressor

# increase the n_estimators (50 -> 64) to see if this improves CV
def get_rf(x_tr, y_tr, x_val, y_val, shape):
    model = RandomForestRegressor(bootstrap=False, max_features=0.3, min_samples_leaf=15, 
                                  min_samples_split=8, n_estimators=64, n_jobs=-1, random_state=42)
    model.fit(x_tr, y_tr)
    
    y_pred = model.predict(x_val)
    y_valid = y_val
    crps = crps_score(y_pred, y_valid, shape=shape)
    
    return model, crps

In [0]:
def get_nn(x_tr, y_tr, x_val, y_val, shape):
    K.clear_session()
    inp = Input(shape = (x_tr.shape[1],))
    x = Dense(2048, input_dim=X.shape[1], activation='elu')(inp)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(1024, activation='elu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='elu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='elu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    
    out = Dense(199, activation='softmax')(x)
    model = Model(inp,out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[])

    
    es = EarlyStopping(monitor='CRPS_score_val', 
                       mode='min',
                       restore_best_weights=True, 
                       verbose=1, 
                       patience=10)

    mc = ModelCheckpoint('best_model.h5',monitor='CRPS_score_val',mode='min',
                                   save_best_only=True, verbose=1, save_weights_only=True)
    
    bsz = 1024
    steps = x_tr.shape[0]/bsz

    model.fit(x_tr, y_tr,callbacks=[CRPSCallback(validation = (x_val,y_val)),es,mc], epochs=200, batch_size=bsz,verbose=1)
    model.load_weights("best_model.h5")
    
    y_pred = model.predict(x_val)
    y_valid = y_val
    crps = crps_score(y_pred, y_valid, shape=shape)

    return model,crps

In [0]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [29]:
%%time

loop = 2
fold = 5

oof_nn = np.zeros([loop, y.shape[0], y.shape[1]])
oof_rf = np.zeros([loop, y.shape[0], y.shape[1]])

models_nn = []
crps_csv_nn = []
models_rf = []
crps_csv_rf = []

s_time = time.time()

for k in range(loop):
    kfold = KFold(fold, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(yards)):
        print("-----------")
        print(f'Loop {k+1}/{loop}' + f' Fold {k_fold+1}/{fold}')
        print("-----------")
        tr_x, tr_y = X[tr_inds], y[tr_inds]
        val_x, val_y = X[val_inds], y[val_inds]
        
        # Train NN
        nn, crps_nn = get_nn(tr_x, tr_y, val_x, val_y, shape=val_x.shape[0])
        models_nn.append(nn)
        print("the %d fold crps (NN) is %f"%((k_fold+1), crps_nn))
        crps_csv_nn.append(crps_nn)
        
        # Train RF
        rf, crps_rf = get_rf(tr_x, tr_y, val_x, val_y, shape=val_x.shape[0])
        models_rf.append(rf)
        print("the %d fold crps (RF) is %f"%((k_fold+1), crps_rf))
        crps_csv_rf.append(crps_rf)
        
        #Predict OOF
        oof_nn[k, val_inds, :] = nn.predict(val_x)
        oof_rf[k, val_inds, :] = rf.predict(val_x)

-----------
Loop 1/2 Fold 1/5
-----------
validation shape 2
Epoch 1/200
18536/18536 [==============================] - 10s 514us/step - loss: 5.4837

Epoch 00001: CRPS_score_val improved from inf to 0.05071, saving model to best_model.h5
Epoch 2/200
18536/18536 [==============================] - 9s 461us/step - loss: 4.9020

Epoch 00002: CRPS_score_val improved from 0.05071 to 0.03388, saving model to best_model.h5
Epoch 3/200
18536/18536 [==============================] - 8s 453us/step - loss: 4.5386

Epoch 00003: CRPS_score_val improved from 0.03388 to 0.02975, saving model to best_model.h5
Epoch 4/200
18536/18536 [==============================] - 9s 461us/step - loss: 4.2067

Epoch 00004: CRPS_score_val improved from 0.02975 to 0.02563, saving model to best_model.h5
Epoch 5/200
18536/18536 [==============================] - 8s 458us/step - loss: 3.8663

Epoch 00005: CRPS_score_val improved from 0.02563 to 0.01886, saving model to best_model.h5
Epoch 6/200
18536/18536 [============

In [30]:
crps_oof_nn = []
crps_oof_rf = []

for k in range(loop):
    crps_oof_nn.append(crps_score(oof_nn[k,...], y))
    crps_oof_rf.append(crps_score(oof_rf[k,...], y))
print("mean crps (NN) is %f"%np.mean(crps_csv_nn))
print("mean crps (RF) is %f"%np.mean(crps_csv_rf))

mean crps (NN) is 0.012735
mean crps (RF) is 0.012927


In [31]:
print("mean OOF crps (NN) is %f"%np.mean(crps_oof_nn))
print("mean OOF crps (RF) is %f"%np.mean(crps_oof_rf))

mean OOF crps (NN) is 0.012736
mean OOF crps (RF) is 0.012927


In [0]:
def weight_opt(oof_nn, oof_rf, y_true):
    weight_nn = np.inf
    best_crps = np.inf
    
    for i in np.arange(0, 1.01, 0.05):
        crps_blend = np.zeros(oof_nn.shape[0])
        for k in range(oof_nn.shape[0]):
            crps_blend[k] = crps_score(i * oof_nn[k,...] + (1-i) * oof_rf[k,...], y_true)
        if np.mean(crps_blend) < best_crps:
            best_crps = np.mean(crps_blend)
            weight_nn = round(i, 2)
            
        print(str(round(i, 2)) + ' : mean crps (Blend) is ', round(np.mean(crps_blend), 6))
        
    print('-'*36)
    print('Best weight for NN: ', weight_nn)
    print('Best weight for RF: ', round(1-weight_nn, 2))
    print('Best mean crps (Blend): ', round(best_crps, 6))
    
    return weight_nn, round(1-weight_nn, 2)

In [33]:
weight_nn, weight_rf = weight_opt(oof_nn, oof_rf, y)

0.0 : mean crps (Blend) is  0.012927
0.05 : mean crps (Blend) is  0.012898
0.1 : mean crps (Blend) is  0.01287
0.15 : mean crps (Blend) is  0.012846
0.2 : mean crps (Blend) is  0.012823
0.25 : mean crps (Blend) is  0.012802
0.3 : mean crps (Blend) is  0.012784
0.35 : mean crps (Blend) is  0.012766
0.4 : mean crps (Blend) is  0.012752
0.45 : mean crps (Blend) is  0.01274
0.5 : mean crps (Blend) is  0.012729
0.55 : mean crps (Blend) is  0.01272
0.6 : mean crps (Blend) is  0.012714
0.65 : mean crps (Blend) is  0.012709
0.7 : mean crps (Blend) is  0.012707
0.75 : mean crps (Blend) is  0.012706
0.8 : mean crps (Blend) is  0.012708
0.85 : mean crps (Blend) is  0.012712
0.9 : mean crps (Blend) is  0.012718
0.95 : mean crps (Blend) is  0.012726
1.0 : mean crps (Blend) is  0.012736
------------------------------------
Best weight for NN:  0.75
Best weight for RF:  0.25
Best mean crps (Blend):  0.012706
